In [1]:
import minsearch
import json

In [2]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

# Flatten the documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # SELECT * WHERE course = 'data-engineering-zoomcamp'; #kind of SQL query for keyword_fields
)

In [6]:
q = "the course has already started, can i still enroll?" #question example

In [7]:
index.fit(documents)

In [8]:
# All these fields como from elastic search and buy default they are all equal to 1.0

boost = {
    "question": 3.0, #the question field is 3 times mr important than the other fields
    'section':0.5 # this filed is 0.5 times less important than the other fields
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI
import os

In [11]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [12]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": "is it too late to join the course?"
         },
    ]
)

In [13]:
response.choices[0].message.content

"The answer to whether it is too late to join a course depends on various factors such as the course enrollment deadlines, the policy of the institution or organization offering the course, and the current date relative to the course start date. \n\nHere are a few steps you can take:\n\n1. **Check the Course Website**: Visit the official website or portal where the course is offered. Look for information regarding enrollment dates, deadlines, and late registration policies.\n\n2. **Contact the Instructor or Administration**: Reach out directly to the course instructor or the administrative office handling course enrollments. They can provide specific information and may offer exceptions or waitlist options.\n\n3. **Consider Online Alternatives**: If the course is no longer accepting students and you still want to learn the material, see if there are similar online courses available on platforms like Coursera, Udacity, or edX.\n\n4. **Self-Study**: If formal enrollment isn't possible, y

In [14]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: {question}

CONTEXT:{context}

""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT:section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [19]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": prompt
         },
    ]
)

In [20]:
print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
